## Libraries

In [27]:
from surprise.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np

In [28]:
ratings = pd.read_csv(r'C:\Users\Gaurisha R Srivastav\OneDrive\IT2 4th Year Study Material\Semester 7\Minor Project\Flipsmart-main\ML APIs\New User API\similar users.csv')

In [29]:
ratings.head(10)

,ITEM,ProductId,Rating,UserId
0,toothpaste,6175005570,5,A39HTATAQ9V7YF
1,whole wheat pasta,1304488608,2,A3JM6GV9MNOF9X
2,white wine,6167061580,4,A1Z513UWSAAO0F
3,grated cheese,1304651104,3,A1WMRR494NWEWV
4,whole weat flour,1304139212,4,A3IAAVS479H7M7
5,eggplant,5357955867,2,AKJHHD5VEH7VG
6,burger sauce,5357955454,2,A1BG8QW55XHN6U
7,french wine,1412759676,5,A22VW0P4VZHDE3
8,cauliflower,6165151329,1,A3V3RE4132GKRO
9,yams,1304139220,4,A327B0I7CYTEJC


### No of Uers and Products

In [30]:
# Total
n_users = ratings.UserId.shape[0]
n_items = ratings.ProductId.shape[0]

print(n_users,n_items)

26375 26375


In [31]:
# Unique
n_users = ratings.UserId.unique().shape[0]
n_items = ratings.ProductId.unique().shape[0]

print(n_users,n_items)

1498 236


### Users who have rated more than 20 times

In [32]:
x = ratings['UserId'].value_counts()>20
x

A2B3YKKXVN28TW     True
A1VK4ALI1QOF5U     True
A2P8AUMTM6O605     True
A3SN6TKIXQI7RP     True
A2MSCG0YX85RL4     True
                  ...  
A2S92B8TOGCIWF    False
A25Q5FLFYVT0K9    False
AL89HJRSN6P06     False
A2R950V3H7DF4U    False
A334RW0AL6LXIC    False
Name: UserId, Length: 1498, dtype: bool

In [33]:
y = x[x].index
y

Index(['A2B3YKKXVN28TW', 'A1VK4ALI1QOF5U', 'A2P8AUMTM6O605', 'A3SN6TKIXQI7RP',
       'A2MSCG0YX85RL4', 'A1MLJHYS35C4X6', 'A2FTLN4PYZE7S6', 'ATPHIBSTV1NY5',
       'A1QWS6V26JZ5OX', 'A2RDR51FRB58LD',
       ...
       'A2DT0SAXS5DIL8', 'A1X1X91COEBHVI', 'A1U6E8IJEATCYM', 'A3590W7FEAGQY4',
       'A225BCNNIG03JT', 'A1UUX9VH9XXX0B', 'A1WSGOA6S3U9OC', 'A2WE4LU82W1U1L',
       'A29AQSCFQ1OZ9R', 'APULYOSZS226A'],
      dtype='object', length=415)

In [34]:
ratings = ratings[ratings['UserId'].isin(y)]

In [35]:
ratings.head

<bound method NDFrame.head of                    ITEM    ProductId  Rating          UserId
5              eggplant   5357955867       2   AKJHHD5VEH7VG
6          burger sauce   5357955454       2  A1BG8QW55XHN6U
7           french wine   1412759676       5  A22VW0P4VZHDE3
8           cauliflower   6165151329       1  A3V3RE4132GKRO
9                  yams   1304139220       4  A327B0I7CYTEJC
...                 ...          ...     ...             ...
26370  dark forest cake  12324565978       2  A294IJJRMVOBSE
26371  mustard hot dogs  12324565979       2  A3FPJWX0GHJCCG
26372      dog pet food  12324565980       1   A5UQ98GXX7T47
26373   cheese sandwich  12324565981       5  A13G2YWZQ3522K
26374   protein fat bar  12324565982       4  A1Q3W2FX8LKH19

[11731 rows x 4 columns]>

In [36]:
no_of_ratings = ratings.groupby('ITEM')['Rating'].count().reset_index()

In [37]:
no_of_ratings.rename(columns={'Rating':'No. of Ratings'},inplace = True)
no_of_ratings

,ITEM,No. of Ratings
0,Parmigiano Reggiano,3
1,african yams,4
2,aged champagne,3
3,aged red wine,3
4,aged white wine,3
...,...,...
224,whole wheat rice,77
225,whole wheat spagetti,4
226,yams,91
227,yogurt cake,130


In [38]:
final_ratings = ratings.merge(no_of_ratings, on='ITEM')
final_ratings

,ITEM,ProductId,Rating,UserId,No. of Ratings
0,eggplant,5357955867,2,AKJHHD5VEH7VG,123
1,eggplant,5357955867,5,A9ZNJ2AXPO652,123
2,eggplant,5357955867,4,A3BMQUL5QATQIW,123
3,eggplant,5357955867,3,A1T8YQ4FFMWQPC,123
4,eggplant,5357955867,5,A1ZY0K46FYOTB6,123
...,...,...,...,...,...
11726,whole wheat spagetti,12324565998,4,AC4343QXC1BCQ,4
11727,oat pasta,12324565999,4,A445U3NN5XBYK,4
11728,oat pasta,12324565999,4,A7SVI5Q1WC3UP,4
11729,oat pasta,12324565999,4,A445U3NN5XBYK,4


In [39]:
# IGNORE THIS CELL
# final_ratings = final_ratings[final_ratings['No. of Ratings']>=90]
# final_ratings.shape

In [40]:
final_ratings.drop_duplicates(['ITEM','UserId'],inplace=True)
final_ratings.shape

(6687, 5)

In [41]:
item_pivot = final_ratings.pivot_table(columns = 'UserId', index = 'ITEM', values = 'Rating')
item_pivot.fillna(0, inplace = True)
item_pivot

UserId,A00205921JHJK5X9LNP42,A024581134CV80ZBLIZTZ,A05492663T95KW63BR75K,A100GYE1W4OXZ8,A10REFE1TW3ZVT,A10ZKDOZOSH219,A11AT6B3912DCU,A11OEJ4R1PDTLN,A11PBM6FRXUJ5Q,A11Q1HQ84F8POH,...,AXG1D956QCMHI,AXVLSEEYTC4Z9,AY25UTTDNJ5SX,AYFZQPNEVSFMD,AYHZNXRPOHMO2,AYI1LDJYGG0ZK,AYM7E80UCJX7I,AZ4FR14IPKML7,AZLE2XFH3JUNG,AZWRTJPN7NXT
ITEM,,,,,,,,,,,,,,,,,,,,,
Parmigiano Reggiano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
african yams,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aged champagne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aged red wine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aged white wine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
whole wheat rice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
whole wheat spagetti,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yams,0.0,0.0,0.0,0.0,5.0,0.0,2.0,5.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0


In [42]:
item_sparse = csr_matrix(item_pivot)
item_sparse

<229x415 sparse matrix of type '<class 'numpy.float64'>'
	with 6687 stored elements in Compressed Sparse Row format>

### Model

In [43]:
model = NearestNeighbors(algorithm='brute')

In [44]:
model.fit(item_sparse)

NearestNeighbors(algorithm='brute')

In [45]:
distances, suggestions = model.kneighbors(item_pivot.loc['baby corn', : ].values.reshape(1,-1), n_neighbors = 6)

In [46]:
print(type(item_pivot.loc['mango cream', : ]))

<class 'pandas.core.series.Series'>


In [47]:
suggestions

array([[ 13, 142, 199, 101, 184, 126]], dtype=int64)

In [48]:
#suggestions for turkey by similar users
for i in range(len(suggestions)):
    print(item_pivot.index[suggestions[i]])

Index(['baby corn', 'mint powder', 'strawberry sparkling water',
       'frozon vegetables mix', 'seasoned herb & pepper',
       'low fat cooking oil'],
      dtype='object', name='ITEM')


In [49]:
print(model.kneighbors)

<bound method KNeighborsMixin.kneighbors of NearestNeighbors(algorithm='brute')>


### Saving the model and the pivot table

In [50]:
#Saving Pivot table
import joblib
pkl_file = open("item_pivot.pkl","wb")
joblib.dump(item_pivot,pkl_file)
pkl_file.close()

In [51]:
#Saving Model
import joblib
pkl_file = open("similarUser.pkl","wb")
joblib.dump(model,pkl_file)
pkl_file.close()

In [52]:
#Load Model and pivot table
data = joblib.load('similarUser.pkl')
pt = joblib.load('item_pivot.pkl')

In [53]:
#Try Outs
a = 'mango cream'
distances, suggestions = data.kneighbors(pt.loc[a, : ].values.reshape(1,-1), n_neighbors = 6)

In [54]:
for i in range(len(suggestions)):
    print(pt.index[suggestions[i]])

Index(['mango cream', 'lemon black tea', 'strawberry sparkling water',
       'mint powder', 'salted toothpaste', 'frozon vegetables mix'],
      dtype='object', name='ITEM')
